In [2]:
import sys
import os
import json
from datetime import datetime

## 아마존 EC2에 올라 갔을때 모듈 위치를 제대로 못잡을수 있으니 아래와 같은 조치를 함.
import numpy as np
import pandas as pd
import pymysql
import statsmodels.api as sm
import statsmodels.formula.api as smf

conn=pymysql.connect(
    host="fermi-master.c4kp2nxu0eer.ap-northeast-2.rds.amazonaws.com",
    user="admin",
    password="fermi1234",
    db="energy_data",
    charset='utf8'
)
params={'startdate':'2018-05-10','year':15,'size':99.5,'weight':1.5,'averagetime':3.4,"scene":"lm_model","type":"month"
        ,"plant":"solar", "contruction":150000000,"investment":180000000,"othercost":25000000,"debt":120000000,"interest":0.05,
        "unredeemed":12321321123,"duration":12,"repayment_method":"cpm","repayment_term":"m","price_index":0.01,"solar_index":-0.08}

/Users/youngji/.local/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [9]:
curs=conn.cursor()
sql="""
select smp_price.date, smp_price.total_price as smp_price, oil_price.wti,elec_supply.supply,price_index.korea_producer, price_index.us_producer from smp_price
    inner join oil_price
		on smp_price.date=oil_price.date
	inner join elec_supply
		on smp_price.date=elec_supply.date
	inner join price_index
		on smp_price.date=price_index.date
        """
curs.execute(sql)
array=np.array(curs.fetchall())
data=pd.DataFrame(data=array[:,1:6],index=array[:,0],columns=["smp_price","oil_price","elec_supply","korea_producer","us_producer"])
data.smp_price=data.smp_price*data.korea_producer
data.oil_price=data.oil_price*data.us_producer
data=data.astype(float)
lm_model=smf.ols(formula = 'smp_price~oil_price+elec_supply', data = data).fit()

# 회귀분석 모델 만들기
def find_fields(tables):
    sql="SELECT * FROM information_schema.columns WHERE Table_Name=%s"
    for i in range(1,len(tables)):
        sql +=" OR Table_Name=%s"
    curs=conn.cursor()
    curs.execute(sql,tables)
    fields=[]
    fields_result=curs.fetchall()
    for i in fields_result:
        fields.append(i[3])
    return fields

def fetch_forecast(scenes,startdate,year):
    fields=find_fields(['elec_forecast'])
    sql="select * from energy_data.elec_forecast"
    curs=conn.cursor()
    curs.execute(sql)
    result=pd.DataFrame(data=np.array(curs.fetchall()),columns=fields)
    result.index=pd.date_range(start=result.date[0],periods=result.shape[0],freq="m")
    result=result.loc[:,scenes]
    start=datetime.strptime(startdate,'%Y-%m-%d')
    result=result.loc[result.index>=start,:]
    final_result=result.iloc[0:year*12+1]
    final_result=final_result.astype(float)
    return(final_result)

def make_scenario(model,forecast):
    pred=model.predict(forecast)
    output=pd.DataFrame({'smp_price':pred})
    output=output.astype(int)
    return(output)

def price_predict(model,startdate,year):
    if model=="lm_model":
        forecast=fetch_forecast(["WEO_450","supply7"],startdate,year)
        forecast=forecast.rename(columns={"WEO_450":"oil_price","supply7":"elec_supply"})
        pred_model=make_scenario(lm_model,forecast)
        return pred_model
    else:
        return 'error'

def index_predict(startdate,year,index):
    data=pd.Series(index=pd.date_range(start=startdate,periods=year*12+1,freq="m"))
    for i in range(0,year):
        data[::12][i]=(1+index)**i
    data=data.interpolate()
    return data

In [11]:
if __name__ == '__main__':
    result=price_predict(params['scene'],params['startdate'],params['year'])
    print(result)

            smp_price
2018-05-31         86
2018-06-30         85
2018-07-31         85
2018-08-31         85
2018-09-30         84
2018-10-31         84
2018-11-30         84
2018-12-31         83
2019-01-31         83
2019-02-28         83
2019-03-31         83
2019-04-30         82
2019-05-31         83
2019-06-30         83
2019-07-31         84
2019-08-31         84
2019-09-30         85
2019-10-31         85
2019-11-30         86
2019-12-31         86
2020-01-31         87
2020-02-29         87
2020-03-31         87
2020-04-30         87
2020-05-31         87
2020-06-30         88
2020-07-31         88
2020-08-31         88
2020-09-30         88
2020-10-31         88
...               ...
2030-12-31         97
2031-01-31         97
2031-02-28         97
2031-03-31         97
2031-04-30         97
2031-05-31         97
2031-06-30         97
2031-07-31         97
2031-08-31         97
2031-09-30         97
2031-10-31         97
2031-11-30         96
2031-12-31         96
2032-01-31